# <center> <span style="color:red">Music recognition </span> </center>

The purpose of this notebook is to take in music spectrograms and to recognize musical genres

## Import 

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_squared_error, accuracy_score

## Import Model Linear and MLP 

In [2]:
import sys  
sys.path.insert(0, '../../python')

from linearModel import linearModel
from mlp import MLPModel
from wrapper import MyMLPRawWrapper

## Create Model 

In [3]:
lm = linearModel()
mlp = MLPModel()

## Algorithme 

In [4]:
def import_images_and_assign_labels(folder, label, X, Y):
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        im = Image.open(image_path)
        im = im.resize((8, 8))
        im = im.convert("RGB")
        im_arr = np.array(im)
        im_arr = np.reshape(im_arr, (8 * 8 * 3))
        X.append(im_arr)
        Y.append(label)

In [5]:
def import_dataset():
    dataset_folder = "C:/Users/Antoine/Desktop/Projet_annuel/projetAnnuel/data/"
    train_folder = os.path.join(dataset_folder, "train")
    test_folder = os.path.join(dataset_folder, "test")

    X_train = []
    y_train = []
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Electro"), [1,0,0], X_train, y_train
    )
    import_images_and_assign_labels(
        os.path.join(train_folder, "Metal"), [0,1,0], X_train, y_train
    )
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Rap"), [0,0,1], X_train, y_train
    )
    
    X_test = []
    y_test = []
    import_images_and_assign_labels(
        os.path.join(test_folder, "Electro"), [1,0,0], X_test, y_test
    )
    import_images_and_assign_labels(
        os.path.join(test_folder, "Metal"), [0,1,0], X_test, y_test
    )
    
    import_images_and_assign_labels(
        os.path.join(test_folder, "Rap"), [0,0,1], X_test, y_test
    )
    
    return (np.array(X_train) / 255.0, np.array(y_train)), \
           (np.array(X_test) / 255.0, np.array(y_test))

In [6]:
def display_value(array):
    print("Sur le dataset de Train")
    cpt = 0
    number = len(array)/3
    for i in array:
        print(cpt," ",np.argmax(i )," ",i)
        cpt+=1
        if(cpt==number):
            cpt=0

In [7]:
def percentage_music_style(array):
    item1 = 0
    item2 = 0
    item3 = 0
    itemGlobal = 0 
    cpt = 0
    for i in array:
        if cpt<10:
            if np.argmax(i) == 0:
                item2 += 10
        elif cpt<20:
            if np.argmax(i) == 1: 
                item1 += 10
        else:
            if np.argmax(i) == 2:
                item3 +=10
        cpt+=1
    itemGlobal = round((item1+item2+item3)/3,2)
        
    print(f"il y a {item2}% de chanson identifié en électro")
    print(f"il y a {item1}% de chanson identifié en métal")
    print(f"il y a {item3}% de chanson identifié en rap")
    print(f"L'accuracy de notre ia est de {itemGlobal}%")

In [ ]:
def my_mean_squared_error(y_true, y_pred):
    return np.sum((y_true - y_pred) ** 2) / len(y_true)

In [1]:
def run_mlp_wrapper():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    wrapped_model = MyMLPRawWrapper([192,4,4,3], True, 0.001, 200000)

    losses = []
    val_losses = []

    for epoch in range(100):
        wrapped_model.fit(X_train, y_train)
        y_pred = wrapped_model.predict(X_train)
        loss = mean_squared_error(y_train, y_pred)
        losses.append(loss)
        val_y_pred = wrapped_model.predict(X_test)
        val_loss = mean_squared_error(y_test, val_y_pred)
        val_losses.append(val_loss)

    plt.plot(losses)
    plt.plot(val_losses)
    plt.show()
    

In [8]:
def run_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat1 = mlp.create_mlp_model([192,4,4,3])
    model1 = resultat1[0]
        
    flattened_dataset_inputs = []
    for p in X_train:
        for i in p:
            flattened_dataset_inputs.append(i)
            
    flattened_dataset_outputs = []
    for p in y_train:
        for i in p:
            flattened_dataset_outputs.append(i)
   
    mlp.train_classification_stochastic_gradient_backpropagation(model1,
                                                                flattened_dataset_inputs,
                                                                flattened_dataset_outputs,
                                                                alpha=1e-3,
                                                                iterations_count=200000)

    predicted_outputs = [mlp.predict_mlp_model_classification(model1, p) for p in X_test]
    

        
    #print(predicted_outputs)
    display_value(predicted_outputs)
    percentage_music_style(predicted_outputs)
    print("predicted_outputs",predicted_outputs)
    mlp.save_mlp_classification(model1)
    
    mlp.free_MLP(resultat1)

In [9]:
run_mlp()

Sur le dataset de Train
0   1   [0.45533132553100586, 0.5404326915740967, -0.06545185297727585]
1   2   [0.33505484461784363, 0.35832440853118896, 0.367656409740448]
2   1   [0.4486907124519348, 0.5059443712234497, 0.020288730040192604]
3   0   [0.417377769947052, 0.35958513617515564, 0.3126053214073181]
4   1   [0.39959457516670227, 0.43666183948516846, 0.19968277215957642]
5   1   [0.40501776337623596, 0.4952942728996277, 0.08352777361869812]
6   1   [0.4769781529903412, 0.5615782737731934, -0.16596093773841858]
7   2   [0.33112460374832153, 0.3589716851711273, 0.36847272515296936]
8   1   [0.43056878447532654, 0.5146377682685852, 0.02188008464872837]
9   1   [0.3534596860408783, 0.3711337447166443, 0.3394043445587158]
10   0   [0.5425788164138794, 0.44930756092071533, -0.058673691004514694]
11   1   [0.47944721579551697, 0.5612778663635254, -0.1599835306406021]
12   1   [0.4784078001976013, 0.529098629951477, -0.07487250119447708]
13   2   [0.27528709173202515, 0.05923059210181236, 

In [12]:
def load_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat2 = mlp.load_mlp_model("../../Save/mlp_classification_192_4_4_3_2021719111216.json")
    model2 = resultat2[0]
        
    predicted_outputs = [mlp.predict_mlp_model_classification(model2, p) for p in X_test]

    display_value(predicted_outputs)
    percentage_music_style(predicted_outputs)
    
    mlp.free_MLP(resultat2)

In [13]:
load_mlp()

Sur le dataset de Train
0   1   [0.4553317427635193, 0.5404325723648071, -0.06545158475637436]
1   2   [0.33505499362945557, 0.3583240807056427, 0.3676570653915405]
2   1   [0.44869107007980347, 0.5059441924095154, 0.020289050415158272]
3   0   [0.4173782467842102, 0.3595852255821228, 0.3126048445701599]
4   1   [0.39959490299224854, 0.43666166067123413, 0.19968298077583313]
5   1   [0.4050179719924927, 0.4952937960624695, 0.08352881669998169]
6   1   [0.4769784212112427, 0.5615781545639038, -0.1659606397151947]
7   2   [0.3311246931552887, 0.3589712381362915, 0.3684735596179962]
8   1   [0.4305693507194519, 0.5146377682685852, 0.021879645064473152]
9   1   [0.35345977544784546, 0.37113332748413086, 0.3394050598144531]
10   0   [0.542579174041748, 0.44930750131607056, -0.05867452919483185]
11   1   [0.47944751381874084, 0.5612779855728149, -0.15998363494873047]
12   1   [0.4784082770347595, 0.5290986895561218, -0.07487290352582932]
13   2   [0.2752878665924072, 0.05923089012503624, 0.7